## Supporting Information.
Borofsky, T. and Feldman, M. 2020. Static environments with limited resources select for multiple foraging strategies rather than conformity in social learners.
# Appendix S2: Solving for Initial Equilibria

In [3]:
#from PIL import Image
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages')
#import os
import numpy as np
import importlib
from numpy import linalg
import helperfuns
from helperfuns import *
import DatFrameFuns
from DatFrameFuns import *
import scipy.stats as scs
import matplotlib as mpl
import matplotlib.pyplot as plt
#importlib.reload(helperfuns)
import pandas as pd
#import sympy as sp
#from sympy import *
#from sympy.solvers import solve
#np.set_printoptions(precision=3, suppress = True)
#import seaborn as sns
# next two libraries are used to flatten list of lists
import functools
import operator
# for formatting tick labels
#from matplotlib.ticker import FormatStrFormatter

#for parallelizing:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool




## First we set up a parameter mesh, as described in Section 5.1

In [2]:

Dvals = np.sort(np.append(np.arange(-2,1.2,0.2),[-0.01,0.01]))
svals = np.sort(np.append(np.arange(0,4.5,0.5), [0.01]))
muvals = np.array([-2, -1, -0.5, -0, 0, 0.5,1,2])
betavals = np.arange(0,1.25,0.25)
df = get_param_grid(Dvals, svals, muvals, betavals)

## We use parallelization to to greatly speed up the dataframe generation. 

In [ ]:

# Parallelize and find equilibria

cores=mp.cpu_count()
# split dataframe over cores, stored in df_split
df_split = np.array_split(df, cores, axis=0)
pool = Pool(cores)

# Run get_1side_df over each dataframe in df_split. 
# get_1side_df iterates 50000 time steps. Stack dataframe (df) parts into one df.

df_out = np.vstack(pool.map(get_1side_df, df_split))
pool.close()
pool.join()
pool.clear()
df_1side = pd.DataFrame(df_out, columns = df.columns)

# fix the column names
colnames = ['mu', 'K', 'pc', 's', 'D', 'beta',
       'u1init', 'u2init', 'buinit', 'r1init', 'r2init', 'u1eq', 'u2eq',
       'bueq', 'r1eq', 'r2eq', 'Weq', 'time', 'reached_eq', 'URstable']
df_1side = df_1side[colnames] # gets rid of the added unnamed columns
#save to csv
df_1side.to_csv('data_1side_1stRound.csv')

## Check the result of the 500,000th iteration to see if it is at equilibrium.

In [ ]:
rtol = 1e-10
atol = 1e-10
u = 1
should0_u1 = df_1side.Weq*df_1side.u1eq - u*(df.K*(df.u1eq + df.D*phi_fun(df.u1eq)) + df.pc)*(1+df.r1eq)
should0_u2 = df_1side.Weq*df_1side.u2eq - u*(df.K*(df.u2eq + df.D*phi_fun(df.u2eq)) + df.pc)*(1+df.r2eq)
should0_bu = df_1side.Weq*df_1side.bueq - u*(df.K*(1 - df.u1eq - df.u2eq - df.D*phi_fun(df.u1eq) - df.D*phi_fun(df.u2eq)) + 1-df.K - df.pc)

reached_eq = np.isclose(should0_u1,0,rtol=rtol, atol = atol)
reached_eq = reached_eq & np.isclose(should0_u2,0,rtol=rtol, atol = atol)
reached_eq = reached_eq & np.isclose(should0_bu,0,rtol=rtol, atol = atol)

df.reached_eq = reached_eq

## Now try to fill in rows that didn't reach equilibrium

In [ ]:
df_correct = df_1side.query('reached_eq==1')
df_fail = df_1side.query('reached_eq==0')
df_fail.time = -1
# from scipy.optimize import fsolve
df_fixed = fsolve_failed_eq(df_fail)

df_1side = df_correct.append(df_fixed)
#df_1side.to_csv('data_1side_fixed.csv')
df_1side.to_csv('data_1side_fixed.csv')

# reflect equilibria in the manner described in the Numerical Analysis, equations 24 - 25.
df_total = reflect_df(df_1side)

# make it look good.
df = df_total.sort_values(by=['mu','beta','s','D'])
df.reset_index(inplace=True, drop=True) 
df = df[colnames]
# There's an issue with setting D = 0. I'm getting weird python e-16 values instead. 
# This can make code difficult later on, so I'm rounding it now
df.D = df['D'].round(6)
# We want later to be able to count how many times iteration failed for each equilibrium
df['iterated'] = df.time!= -1
df.to_csv('data.csv')

In [ ]:
# We have a lot of repeat equilibria in the dataframe df because many of the initial points go to the same equilibrium.
# We now extract the unique equilibria for each parameter combination.
df_main = df[df.reached_eq==1]
unique_eq = get_UniqueEquilibria(df_main)
diff = np.abs(unique_eq.u1eq - unique_eq.u2eq)
unique_eq['difference'] = diff
unique_eq.to_csv('UniqueEquilibriaDF.csv')

# Check Stability

There are many cases of $C_D = 0$ that need to be resolved

In [ ]:
df = pd.read_csv('UniqueEquilibriaDF.csv')
df = df.drop(columns='Unnamed: 0')
y_neg_invades = df.C_D < 0
df['y_neg_invades'] = y_neg_invades
y_pos_invades = df.C_D > 0
df['y_pos_invades'] = y_pos_invades

In [ ]:

def Check_Stable_D(row):
# We have a bunch of points with C_D = 0. Check_Stable_D checks if they're actually stable

    row = row.copy() # explicitly tell python i'm making a copy so i don't get the warning
    
    # get vectors of post perturb values of all the frequencies
    umat,ymat,rmat = Perturb(row) # Perturb is a function defined below
    umat = np.array(umat); ymat = np.array(ymat); rmat = np.array(rmat)
    # check stability to increase in D
    
    dD =  0.01
    n = len(umat[0]) 
    y_pos_invades = False # default is that it is stable
    if row.D <= 0.99:
        for i in range(0,n):
            result = NextGen(umat[:,i],[0,0,0],ymat[:,i],rmat[:,i], 
                             row.D, row.K,row.pc,row.beta,
                             deltas = [dD, 0, 0], eta=1)
            yvec = result[2]
            y = sum(yvec)
            if y > 0.01: 
                y_pos_invades = True
                break
    
    dD = -0.01
    y_neg_invades = False # default is that it is stable
    #check stability to decrease in D
    if row.D >= -1.99: # otherwise it's stable because D can't go down further
        for i in range(0,n):
            result = NextGen(umat[:,i],[0,0,0],ymat[:,i],rmat[:,i], 
                             row.D, row.K,row.pc,row.beta,
                             deltas = [dD, 0, 0], eta=1)
            yvec = result[2]
            y = sum(yvec)
            if y > 0.01: 
                y_neg_invades = True 
                break
    
    row.loc['y_pos_invades'] = y_pos_invades
    row.loc['y_neg_invades'] = y_neg_invades
    return(row)

def Perturb(row):
# After the [u1,u2,bu,r1,r2] eq is perturbed with the addition of the a or b allele, get new frequencies
# perturb by a magnitude of 0.01... so |dr1| = |dr2| = |du| = 0.01, and either |dx| or |dy| = 0.01

    # use dyvec here but dxvec could work too
    uvec = [row.u1eq, row.u2eq, row.bueq]
    rvec = [row.r1eq, row.r2eq]
    # get new post-perturb vectors
    
    # no need to check if valid
    dy = 0.01
    dy1vec = np.array([0.05, 0.3,0.48, 0.08])*dy
    dy2vec = np.array([0.1,0.6,0.5,0.9])*dy
    dbyvec = dy - dy1vec - dy2vec
    which_y = [0,1,2,3]
    
    #dr... prune r + dr values that are invalid
    
    dr1=0 #the default is not changing r1
    if rvec[0]>0:
        dr1 = np.array([-0.01,0.01])
        check_r1 = (rvec[0] + dr1 > 0)&(rvec[0] + dr1 < 1)
        dr1 = dr1[check_r1]
    dr2 = 0
    if rvec[1]>0:
        dr2 = np.array([-0.01,0.01])
        check_r2 = (rvec[1] + dr2 > 0) & (rvec[1] + dr2 < 1)
        dr2 = dr2[check_r2]
    
    # du perturbations... look similar to x perturbs, but opposite direction
    du = -dy
    du1vec = np.array([0.015, 0.29,0.49, 0.07])*du
    du2vec = np.array([0.11,0.49,0.51,0.89])*du
    dbuvec = du - du1vec - du2vec
    which_u = [0,1,2,3]
    for i in which_u:
        duvec = [du1vec[i],du2vec[i],dbuvec[i]]
        [du1vec[i],du2vec[i],dbuvec[i]] = Perturb_EdgeCase(uvec,duvec)
    
    # now find all the combinations
    which_y_mesh, which_u_mesh, DR1, DR2 = np.meshgrid(which_y, which_u, dr1, dr2)
    [which_y, which_u, dr1, dr2] = [np.ndarray.flatten(item) for 
                                    item in [which_y_mesh, which_u_mesh, DR1, DR2]]
    y1 = dy1vec[which_y]; y2 = dy2vec[which_y]; by = dbyvec[which_y]
    u1 = uvec[0] + du1vec[which_u]
    u2 = uvec[1] + du2vec[which_u]
    bu = uvec[2] + dbuvec[which_u]
    r1 = rvec[0] + dr1
    r2 = rvec[1] + dr2
    
    
    return([u1,u2,bu],[y1,y2,by],[r1,r2])

def Perturb_EdgeCase(uvec,duvec):
    #recursively checks for edge cases so i don't get an invalid frequency. Adjusts duvec if needed
    
    # make sure using numpy arrays
    du = sum(duvec)
    uvec = np.array(uvec); duvec = np.array(duvec);
    # find locations of edge cases
    edge_bool = uvec + duvec <= 0
    
    n = sum(edge_bool)
    if n>0:
        duvec[edge_bool] = -uvec[edge_bool] +0.00001 # so not at exactly 0
        du_remain = du - sum(duvec)
        duvec[~edge_bool] = duvec[~edge_bool] + (1/np.float(3-n))*du_remain
        
        # make sure that we didn't cause a different frequency to be negative:
        return(Perturb_EdgeCase(uvec,duvec))

    else:
        return(duvec)
    

def get_stable_check(df):
    # this function is just a wrapper to be used with pool.map that takes a dataframe as an input and
    # runs Check_Stable_D over the data frame
    df = df.apply(lambda row: Check_Stable_D(row), axis = 1)
    return(df)

In [ ]:

cols = df.columns
df_normal = df.query('C_D!=0')
df_weird = df.query('C_D==0')


In [ ]:
cores=mp.cpu_count()
df_split = np.array_split(df_weird, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(get_stable_check, df_split))
pool.close()
pool.join()
pool.clear()
df_fixed = pd.DataFrame(df_out, columns = cols) # restoring original column names
df_total = df_normal.append(df_fixed)
df_total.to_csv('UniqueEquilibriaDF.csv')

# Next, go to SensitivityAnalysis.Rmd to see analysis of these equilibria